In [1]:
import numpy as np
import os
import torch
import torchvision
import cv2
from torchvision import models
from torchvision import transforms
from  PIL import Image


In [2]:
resnet18 = models.resnet18(pretrained=True)

In [3]:
resnet34 = models.resnet34(pretrained=True)

In [6]:
resnet50 = models.resnet50(pretrained=True)

In [4]:
resnet101 = models.resnet101(pretrained=True)

In [5]:
resnet152 = models.resnet152(pretrained=True)

In [8]:
# choose the model.
resnet = resnet18

In [9]:
resnet

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [10]:
preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406], 
            std=[0.229, 0.224, 0.225]
        )])

In [13]:
path = r'C:\Users\richa\Desktop\projects\Bristol uni porjects\Univecity projects\Applied Data Science\uob_image_set'

In [234]:
# def load_images_from_folder(folder, end, as_tensor = False):

#     images = []
#     files = folders = 0
#     count= 0
#     dirnames = [] 

#     num_img = []

#     for _, dirnames, filenames in os.walk(folder):

 

#         if dirnames != []:
#             subfolders = dirnames
#         current_path = os.path.join(folder, subfolders[count])
#         num_img.append(len(os.listdir(current_path)))
#         for filename in os.listdir(current_path):
#             img = cv2.imread(os.path.join(current_path, filename))
#             if img is not None:
#                 if as_tensor:
#                     img = torch.from_numpy(img * 1.0)
#                     # img = torch.from_numpy(img).float()
#                     images.append(img)
#                 else: 
#                     img = Image.fromarray(img)
# #                     print(type(img))
#                     images.append(img)

 

#         count += 1
        
#         if count % 10 == 0:
#             print('number of folder done =', count, 'total number of images so far =', len(images))
            
#         if count  == end:
#             return images, num_img
#         # only do 300 out of the 1500 folders...is too big otherwise
#         # if count == 100:
#         #     return images
#     return images, num_img

In [17]:
def load_images_from_folder(folder, end, as_tensor = False):

    images = []
    files = folders = 0
    count= 0
    dirnames = [] 

    num_img = []

    for _, dirnames, filenames in os.walk(folder):

 

        if dirnames != []:
            subfolders = dirnames
        current_path = os.path.join(folder, subfolders[count])
        num_img.append(len(os.listdir(current_path)))
        for filename in os.listdir(current_path):
            img = Image.open(os.path.join(current_path, filename))
            if img is not None:
                if as_tensor:
                    img = torch.from_numpy(img * 1.0)
                    images.append(img)
                else: 
                    images.append(img)

 

        count += 1
        
        if count % 10 == 0:
            print('number of folder done =', count, 'total number of images so far =', len(images))
            
        if count  == end:
            return images, num_img
        # only do 300 out of the 1500 folders...is too big otherwise
        # if count == 100:
        #     return images
    return images, num_img


def show_image(PIL_img):
    PIL_img.show()

In [ ]:
def load_images_from_folder(folder, end, as_tensor = False):
        
    images = []
    files = folders = 0
    # count denotes the number of folders to search through
    count= 0
    dirnames = [] 

 

    for _, dirnames, filenames in os.walk(folder):

 

        if dirnames != []:
            subfolders = dirnames
        current_path = os.path.join(folder, subfolders[count])  
        for filename in os.listdir(current_path):
            img = Image.open(os.path.join(current_path, filename))
            if img is not None:
                if as_tensor:
                    img = torch.from_numpy(img * 1.0)
                    # img = torch.from_numpy(img).float()
                    images.append(img)
                else: 
                    images.append(img)

 

        count += 1
        
        if count  == end:
            return images
        
    return images


def show_image(PIL_img):
    PIL_img.show()

In [22]:
dataset, num_img_list = load_images_from_folder(path, 3)

In [23]:
len(dataset)

13

In [24]:
print(num_img_list), print(sum(num_img_list))

[5, 4, 4]
13


(None, None)

In [26]:
list_input_tensor = []
list_input_batch = []
num_img = sum(num_img_list)
for item in range(num_img - 1):
    input_tensor = preprocess(dataset[item])
    input_batch = input_tensor.unsqueeze(0)
    list_input_tensor.append(input_tensor)
    list_input_batch.append(input_batch)
    

In [27]:
torch.cuda.is_available()

True

In [28]:
for param in resnet.parameters():
    param.requires_grad = False
    
# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = resnet.fc.in_features
resnet.fc = torch.nn.Linear(num_ftrs, 3)


In [29]:
for i in range(len(list_input_batch)):
    if torch.cuda.is_available():
        input_batch = list_input_batch[i].to('cuda')
        resnet.to('cuda')
    
    with torch.no_grad():
        output = resnet(input_batch)
    print(output)

tensor([[-0.7197,  0.4163, -0.1825]], device='cuda:0')
tensor([[-0.7517,  0.4607, -0.1521]], device='cuda:0')
tensor([[-0.7484,  0.4555, -0.1553]], device='cuda:0')
tensor([[-0.7220,  0.4278, -0.1583]], device='cuda:0')
tensor([[-0.7416,  0.4681, -0.1596]], device='cuda:0')
tensor([[-0.7488,  0.4658, -0.1630]], device='cuda:0')
tensor([[-0.7231,  0.4040, -0.1827]], device='cuda:0')
tensor([[-0.7260,  0.4446, -0.1814]], device='cuda:0')
tensor([[-0.7261,  0.4534, -0.1884]], device='cuda:0')
tensor([[-0.6819,  0.4259, -0.1550]], device='cuda:0')
tensor([[-0.7501,  0.4809, -0.1387]], device='cuda:0')
tensor([[-0.6775,  0.4692, -0.1547]], device='cuda:0')


In [245]:
output_array = []
for i in range(len(list_input_batch)):
    if torch.cuda.is_available():
        input_batch = list_input_batch[i].to('cuda')
        resnet.to('cuda')
    
    with torch.no_grad():
        output = resnet(input_batch)
        cup_tensor = output.cpu()
        
    output_array.append(cup_tensor[0].tolist())
#     output_array.append(np.asarray(cup_tensor[0]))
#     print(np.asarray(cup_tensor[0]))
output_array = np.asarray(output_array)
display(output_array)

array([[-0.29384795, -0.35663211,  0.02612396],
       [-0.30435032, -0.3663305 ,  0.02636867],
       [-0.27115431, -0.37009948,  0.06807595],
       [-0.29218262, -0.38294515,  0.05986913],
       [-0.26686826, -0.36159807,  0.07712689],
       [-0.27379483, -0.35311711,  0.04263894],
       [-0.26608825, -0.34815207,  0.03915934],
       [-0.27748123, -0.3415657 ,  0.01566445],
       [-0.30066267, -0.35829026,  0.05926574],
       [-0.25781876, -0.3173559 ,  0.08615623],
       [-0.32359609, -0.35515097,  0.06267235],
       [-0.27371293, -0.36206087,  0.06394663]])

In [246]:
import json
import matplotlib.pyplot as plt
import itertools
import random
from scipy.interpolate import *
from mpl_toolkits.mplot3d import axes3d

In [247]:
import pylab
import numpy 
import ipympl

fig = plt.figure()
ax = plt.axes(projection="3d")

img_counter = 0
for n in num_img_list:
    r = lambda: random.randint(0,255)
    random_color = '#%02X%02X%02X' % (r(),r(),r())
    ax.scatter(output_array[img_counter:img_counter + n,0], output_array[img_counter:img_counter + n,1], output_array[img_counter:img_counter + n,2],color=random_color,s=20)
    img_counter += n
plt.show()  

ax.set_xlabel('X Axes')
ax.set_ylabel('Y Axes')
ax.set_zlabel('Frame Axes')

plt.show()

# length = data.shape[0]
# width = data.shape[1]
# x, y = np.meshgrid(np.arange(length), np.arange(width))

# fig = plt.figure()
# ax = fig.add_subplot(1,1,1, projection='3d')
# ax.plot_surface(x, y, data)
# plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [248]:
fig = plt.figure()
img_counter = 0
for n in num_img_list:
    r = lambda: random.randint(0,255)
    random_color = '#%02X%02X%02X' % (r(),r(),r())
    plt.plot(output_array[img_counter:img_counter + n,0], output_array[img_counter:img_counter + n,1], 'o',color=random_color)
    img_counter += n
plt.show()            

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [249]:
fig = plt.figure()
img_counter = 0
for n in num_img_list:
    r = lambda: random.randint(0,255)
    random_color = '#%02X%02X%02X' % (r(),r(),r())
    plt.plot(output_array[img_counter:img_counter + n,0], output_array[img_counter:img_counter + n,2], 'o',color=random_color)
    img_counter += n
plt.show()            

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [250]:
fig = plt.figure()
img_counter = 0
for n in num_img_list:
    r = lambda: random.randint(0,255)
    random_color = '#%02X%02X%02X' % (r(),r(),r())
    plt.plot(output_array[img_counter:img_counter + n,1], output_array[img_counter:img_counter + n,2], 'o',color=random_color)
    img_counter += n
plt.show()            

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [31]:
fig = plt.figure()
plt.plot(output_array[:4,0], output_array[:4,1], 'o',color='r')
plt.plot(output_array[4:,0], output_array[4:,1], 'o',color='b')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [32]:
fig = plt.figure()
plt.plot(output_array[:4,0], output_array[:4,2], 'o',color='r')

plt.plot(output_array[4:,0], output_array[4:,2], 'o',color='b')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [33]:
fig = plt.figure()
plt.plot(output_array[:4,1], output_array[:4,2], 'o',color='r')
plt.plot(output_array[4:,1], output_array[4:,2], 'o',color='b')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [132]:
import pylab
import numpy 
import ipympl

# %matplotlib widget
# %matplotlib notebook

# x = [item[0] for item in output_array]
# y = [item[1] for item in output_array]
# z = [item[2] for item in output_array]
# # output_array[:,0], output_array[:,1], output_array[:,2]

# output_array_1 = numpy.array(output_array)
# print(output_array_1)

fig = plt.figure()
ax = plt.axes(projection="3d")
ax.scatter(output_array[:4,0], output_array[:4,1], output_array[:4,2],color='r',s=3)
ax.scatter(output_array[4:8,0], output_array[4:8,1], output_array[4:8,2],color='y',s=3)
ax.scatter(output_array[8:,0], output_array[8:,1], output_array[8:,2],color='b',s=3)
ax.set_xlabel('X Axes')
ax.set_ylabel('Y Axes')
ax.set_zlabel('Frame Axes')

plt.show()

# length = data.shape[0]
# width = data.shape[1]
# x, y = np.meshgrid(np.arange(length), np.arange(width))

# fig = plt.figure()
# ax = fig.add_subplot(1,1,1, projection='3d')
# ax.plot_surface(x, y, data)
# plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …